In [74]:
import pandas as pd

In [75]:
df = pd.read_csv('buildings.csv')

In [76]:
df.iloc[:, 3]

,residential_complex
0,ЖК «Кронфорт.Центральный»
1,ЖК «Шепилевский (Shepilevskiy)»
2,ЖК «Кронфорт.Центральный»
3,ЖК «Лермонтовский 54»
4,ЖК «Avant (Авант)»
...,...
1414,ЖК «Ассамблея»
1415,ЖК «PLUS Пулковский (Плюс Пулковский)»
1416,ЖК «MONODOM ART (МОНОДОМ АРТ)»
1417,ЖК «Ручьи»


In [77]:
import pandas as pd
from google.colab import userdata

In [78]:
import requests
import uuid

def get_token(auth_token, scope='GIGACHAT_API_PERS'):
    # создадим идентификатор
    rq_uid = str(uuid.uuid4())
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
    # заголовки
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }

    # тело запроса
    payload = {
        'scope': scope
    }

    try:
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return -1

In [79]:
response = get_token(userdata.get('auth'))
if response != 1:
  giga_token = response.json()['access_token']

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [80]:
# формирование запросов и отправление в чат
import json
df['answer'] = ''
for idx, row in df.iterrows():
  url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"
  headers = {
          'Content-Type': 'application/json',
          'Accept': 'application/json',
          'Authorization': f'Bearer {giga_token}'
  }
  payload = json.dumps({
          "model": "GigaChat",
          "messages": [
              {
                  "role": "user",
                  "content": f'''Тут представлена информация о квартире
                  price (цена)
                  floor_current  (этаж квартиры)
                  floor_total (количество этажей в доме)
                  hc_class (класс жк)
                  description (описание квартиры и жк)
                  views (количество просмотров объявления)
                  min_metro (количество минут до метро)
                  online_booking (доступно ли онлайн бронирование)
                  height (высота потолков)
                  total_area (полная площадь квартиры)
                  living_area (площадь для жизни в квартире)
                  rooms (количество комнат, 0-студия, 1-однокомнатная и тд)
                  available (сдан ли дом)
                  ready (есть ли отделка)
                  is_city (находится ли в городе(True) или на переферии/городе спутнике(False))
                  popular_developer (является девелопером известным)
                  elevators (количество лифтов в доме)
                  court (есть ли внутренний двор)
                  entrance (является ли входная группа преимуществом)
                  parking (есть ли парковка)
                  parks  (являются ли парки преимуществом)
                   Напиши цифру подходящего кластера в виде ответа, ответом должна быть одна цифра в стандартном шрифте и больше ничего, не поясняй выбор.
                   Пример ответа: 0.
                   Вот информация: {row}.
                   То, что точно должно соблюдаться при выборе кластера (необходимо точное слодование, первично)
                    rooms: 0 или 1 для кластера 4,
                   2 или 3 для кластера 3, от 0 до 2 для кластера 0
                  hc_class: "Комфорт" для 0 и 3, "Бизнес/Премиум" для 1 и 4
                  min_metro: Самый короткий для 4 и 2, средний для остальных
                  price_per_m2: Высокий для 1 и 4, средний для 0 и 2, ниже для 3. Если есть сомнения, куда отнести квартиру, это кластер 0.
                  Общая характеристика (вторична) Кластер 0: застройщик, метро, проект, вид, школа, квартир, окна, двор, цены, расположение.
                  Работа: Найди оптимальный объект для вложений или безопасной покупки с ясными преимуществами.
                  Кластер 3: район, новый, магазины, метро, школа, красивый, инфраструктура, хорошая, тихий, уютный.
                  Работа: Найди уже сформированный, гармоничный район для жизни "здесь и сейчас.

                  Кластер 2: место, отличное, отличное место, хорошее, жизни, хорошее место, метро, тихое, прекрасное, место жизни.
                  Работа: Найди безупречное место с идеальным балансом доступности и качества среды

                  Кластер 1: комплекс, жилой, жилой комплекс, красивый, дворы, современный, школа, нравится, сад, инфраструктура.
                  Работа: Найди современный, технологичный жилой комплекс как законченный продукт.

                  Кластер 4: отель, персонал, чисто, номере, номера, номер, удобное, расположение, удобное расположение, понравилось.
                  Работа: Обеспечь безупречно комфортное, чистое временное проживание без хлопот, необходима не покупка квартиры, а временное проживание или покупка апартаментов в качестве инвестиции.



'''  # Содержание сообщения
              }
          ],
          "stream": False,
          "update_interval": 0
      })
  response = requests.request("POST", url, headers=headers, data=payload, verify=False)
  # запись ответов
  df.at[idx, 'answer'] = response.json()['choices'][0]['message']['content']
  print(response.json()['choices'][0]['message']['content'])

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


3


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


1


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

4


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


3


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


3


In [81]:
# запись в файл
df.to_csv('buildings_classes.csv', index=False, encoding='utf-8-sig')

In [83]:
df['answer'].value_counts()

,count
answer,
3,680
4,347
0,245
1,101
"Генеративные языковые модели не обладают собственным мнением — их ответы являются обобщением информации, находящейся в открытом доступе. Чтобы избежать ошибок и неправильного толкования, разговоры на чувствительные темы могут быть ограничены.",18
"Как и любая языковая модель, GigaChat не обладает собственным мнением и не транслирует мнение своих разработчиков. Ответ сгенерирован нейросетевой моделью, обученной на открытых данных, в которых может содержаться неточная или ошибочная информация. Во избежание неправильного толкования, разговоры на некоторые темы временно ограничены.",15
"К сожалению, иногда генеративные языковые модели могут создавать некорректные ответы, основанные на открытых источниках. Во избежание неправильного толкования, ответы на вопросы, связанные с чувствительными темами, временно ограничены. Благодарим за понимание.",13
